In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.tree import DecisionTreeClassifier, export_text

from scipy.spatial.distance import cdist


In [2]:
K_CLUSTERS = 3
P = 1
Q = 1
RANDOM_STATE = 42
SAMPLE_SIZE = 1000
MAX_TREE_DEPTH = 5
MIN_NODE_SIZE = 20


In [ ]:
def load_data():

    df = pd.read_csv(
        r'D:\Thesis\Fair_explainable_cluster_updated_2nd_feb\data\bank-full.csv'
    )

    df['sensitive'] = df['marital'].apply(
        lambda x: 1 if x == 'married' else 0
    )

    drop_cols = ['marital', 'sensitive', 'y']
    X_raw = df.drop(columns=drop_cols)

    sensitive = df['sensitive'].values

    if SAMPLE_SIZE < len(X_raw):
        idx = np.random.choice(len(X_raw), SAMPLE_SIZE, replace=False)
        X_raw = X_raw.iloc[idx].reset_index(drop=True)
        sensitive = sensitive[idx]

    categorical_cols = X_raw.select_dtypes(include=['object']).columns
    numeric_cols = X_raw.select_dtypes(include=['number']).columns

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

    X_processed = preprocessor.fit_transform(X_raw)

    feature_names = preprocessor.get_feature_names_out()

    return X_processed, sensitive, feature_names


In [4]:
class TreeNode:

    def __init__(self, indices, depth=0):
        self.indices = indices
        self.left = None
        self.right = None
        self.depth = depth


In [5]:
def build_tree(X, indices, depth=0):

    if depth >= MAX_TREE_DEPTH or len(indices) <= MIN_NODE_SIZE:
        return TreeNode(indices, depth)

    dim = depth % X.shape[1]
    values = X[indices, dim]
    median = np.median(values)

    left_indices = [i for i in indices if X[i, dim] <= median]
    right_indices = [i for i in indices if X[i, dim] > median]

    if len(left_indices) == 0 or len(right_indices) == 0:
        return TreeNode(indices, depth)

    node = TreeNode(indices, depth)
    node.left = build_tree(X, left_indices, depth+1)
    node.right = build_tree(X, right_indices, depth+1)

    return node


In [6]:
def extract_fairlets_from_node(indices, sensitive):

    reds = [i for i in indices if sensitive[i] == 1]
    blues = [i for i in indices if sensitive[i] == 0]

    fairlets = []

    while len(reds) >= Q and len(blues) >= P:
        r = [reds.pop() for _ in range(Q)]
        b = [blues.pop() for _ in range(P)]
        fairlets.append(r + b)

    leftovers = reds + blues

    return fairlets, leftovers


In [7]:
def tree_fairlet_decomposition(node, X, sensitive):

    if node.left is None and node.right is None:
        fairlets, leftovers = extract_fairlets_from_node(
            node.indices, sensitive
        )
        return fairlets, leftovers

    left_fairlets, left_leftovers = tree_fairlet_decomposition(
        node.left, X, sensitive
    )

    right_fairlets, right_leftovers = tree_fairlet_decomposition(
        node.right, X, sensitive
    )

    combined_indices = left_leftovers + right_leftovers

    new_fairlets, final_leftovers = extract_fairlets_from_node(
        combined_indices, sensitive
    )

    total_fairlets = left_fairlets + right_fairlets + new_fairlets

    return total_fairlets, final_leftovers


In [8]:
def compute_fairlet_centers(X, fairlets):

    centers = []

    for fl in fairlets:
        pts = X[fl]
        D = cdist(pts, pts)
        medoid_index = np.argmin(D.sum(axis=1))
        centers.append(fl[medoid_index])

    return np.array(centers)


In [9]:
def kmedoids(X, k, max_iter=100):

    np.random.seed(RANDOM_STATE)
    n = X.shape[0]
    medoids = np.random.choice(n, k, replace=False)

    for _ in range(max_iter):

        distances = cdist(X, X[medoids])
        labels = np.argmin(distances, axis=1)

        new_medoids = []

        for i in range(k):
            cluster_points = np.where(labels == i)[0]

            if len(cluster_points) == 0:
                new_medoids.append(medoids[i])
                continue

            cluster_distances = cdist(
                X[cluster_points], X[cluster_points]
            )
            best = cluster_points[np.argmin(cluster_distances.sum(axis=1))]
            new_medoids.append(best)

        new_medoids = np.array(new_medoids)

        if np.all(new_medoids == medoids):
            break

        medoids = new_medoids

    return labels, medoids


In [10]:
def assign_labels(fairlets, center_labels, n):

    labels = np.zeros(n, dtype=int)

    for fl, lab in zip(fairlets, center_labels):
        for idx in fl:
            labels[idx] = lab

    return labels


In [13]:
def fairness_metrics(labels, sensitive):

    K = len(np.unique(labels))
    balances = []
    violations = 0

    global_ratio = sensitive.mean()
    dp_gaps = []

    for k in range(K):
        mask = labels == k
        group = sensitive[mask]

        n0 = np.sum(group == 0)
        n1 = np.sum(group == 1)

        if max(n0, n1) == 0:
            continue

        balance = min(n0, n1) / max(n0, n1)
        balances.append(balance)

        if balance < 1.0:   # p:q = 1:1
            violations += 1

        cluster_ratio = group.mean()
        dp_gaps.append(abs(cluster_ratio - global_ratio))

    return {
        "avg_balance": np.mean(balances),
        "violation_rate": violations / K,
        "avg_dp_gap": np.mean(dp_gaps)
    }


In [14]:
def explainability_metrics(X, labels):

    clf = DecisionTreeClassifier(max_depth=4, random_state=42)
    clf.fit(X, labels)

    return {
        "tree_fidelity": clf.score(X, labels),
        "tree_depth": clf.get_depth(),
        "tree_leaves": clf.get_n_leaves()
    }


In [15]:
def prototype_quality(X, labels):

    K = len(np.unique(labels))
    distances = []

    for k in range(K):
        pts = X[labels == k]
        center = pts.mean(axis=0)
        d = np.linalg.norm(pts - center, axis=1)
        distances.append(np.mean(d))

    return np.mean(distances)


In [16]:
def main():

    print("Loading data...")
    X, sensitive = load_data()

    print("Building spatial tree...")
    root = build_tree(X, list(range(len(X))))

    print("Running tree-based fairlet decomposition...")
    fairlets, leftovers = tree_fairlet_decomposition(
        root, X, sensitive
    )

    print("\nFAIRLET SUMMARY")
    print("----------------")
    print("Total fairlets:", len(fairlets))
    print("Total assigned points:", sum(len(fl) for fl in fairlets))
    print("Leftovers (unassigned):", len(leftovers))

    # -------------------------------------------------
    # Compute medoids of fairlets
    # -------------------------------------------------
    centers = compute_fairlet_centers(X, fairlets)

    center_labels, medoids = kmedoids(
        X[centers],
        K_CLUSTERS
    )

    # -------------------------------------------------
    # Assign fairlet labels to original points
    # -------------------------------------------------
    labels = np.zeros(len(X), dtype=int)

    for fl, lab in zip(fairlets, center_labels):
        for idx in fl:
            labels[idx] = lab

    # -------------------------------------------------
    # Assign leftover points to nearest cluster
    # -------------------------------------------------
    if len(leftovers) > 0:

        # Get actual medoid coordinates
        medoid_points = X[centers][medoids]

        distances = cdist(X[leftovers], medoid_points)
        leftover_labels = np.argmin(distances, axis=1)

        for idx, lab in zip(leftovers, leftover_labels):
            labels[idx] = lab

        print("Leftovers assigned to nearest clusters.")

    # -------------------------------------------------
    # EVALUATION
    # -------------------------------------------------

    print("\nCLUSTERING QUALITY")
    print("------------------")
    print("Silhouette:", silhouette_score(X, labels))
    print("Davies-Bouldin:", davies_bouldin_score(X, labels))

    print("\nFAIRNESS")
    print("--------")
    fm = fairness_metrics(labels, sensitive)
    for k, v in fm.items():
        print(k, ":", v)

    print("\nEXPLAINABILITY")
    print("--------------")
    em = explainability_metrics(X, labels)
    for k, v in em.items():
        print(k, ":", v)

    print("\nPROTOTYPE QUALITY")
    print("-----------------")
    print("avg_distance:", prototype_quality(X, labels))


In [17]:
main()


Loading data...
Building spatial tree...
Running tree-based fairlet decomposition...

FAIRLET SUMMARY
----------------
Total fairlets: 327
Total assigned points: 654
Leftovers (unassigned): 346
Leftovers assigned to nearest clusters.

CLUSTERING QUALITY
------------------
Silhouette: 0.028280387525811638
Davies-Bouldin: 4.354095400585686

FAIRNESS
--------
avg_balance : 0.5052515300791164
violation_rate : 1.0
avg_dp_gap : 0.02373685542593103

EXPLAINABILITY
--------------
tree_fidelity : 0.753
tree_depth : 4
tree_leaves : 16

PROTOTYPE QUALITY
-----------------
avg_distance: 3.0956377337096885


C:\Users\91773\AppData\Local\Temp\ipykernel_20716\29975366.py:21: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = X_raw.select_dtypes(include=['object']).columns
